# Persistence and Streaming

Based on [**this tutorial**](https://learn.deeplearning.ai/courses/ai-agents-in-langgraph/lesson/5/persistence-and-streaming)

# Principles

When building agents, they're often working on **longer running tasks**.

For this type of tasks, there are two important concepts:
- **persistence**,
- **streaming**.

> **Persistence** lets you keep around the state of an agent at a particular point in time. This lets you:
> - go back to that state,
> - resume in that state in future interactions.

> With **streaming**, you can emit a list of signals of what's going on at that exact moment. So, for long running applications, you know exactly what the agent is doing.

In this notebook, we will see these concepts in action.

# Setup

In [2]:
from dotenv import load_dotenv

In [3]:
_ = load_dotenv()

# Imports and Basic Implementation

In [17]:
import operator
from typing import Annotated, TypedDict

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import AnyMessage, HumanMessage, SystemMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import END, StateGraph
from rich import print as rprint  # Enhance pretty printing for outputs

In [5]:
# Instanciate Search Tool
tool = TavilySearchResults(max_results=2)

In [6]:
# Define AgentState
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

# Persistence

In order to deal with **persistence**, we will use what's called a **checkpointer** into LangGraph.

It basically **checkpoints the state between every node**.

Here, we'll make use of a `SqliteSaver`, and use it **"in memory"**. This basically means that, when leaving the notebook, memory will be erased. Of course, we could keep track of the previous states with connecting this to an external database (some checkpointers, for example, use *Postgres* or *Redis*). We won't do it as this is a POC and we only want to illustrate the concept.

In [7]:
from langgraph.checkpoint.sqlite import SqliteSaver

In [8]:
memory = SqliteSaver.from_conn_string(":memory:")

It's then really easy to incorporate it within our `Agent` class.

In [9]:
# Define Agent
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):  # 👈 REFERENCE TO CHECKPOINTER
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)  # 👈 REFERENCE TO CHECKPOINTER
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

> **NOTE**
>
> For data persistence, it can also be made use of other databases, or [**Redis**](https://redis.io/) for example.

In [10]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-3.5-turbo")  # 👈 Change to -4o for POC
abot = Agent(model, [tool], system=prompt, checkpointer=memory)  # 👈 Passing the memory object previously defined

# Streaming

There are mainly two things that we might care about streaming:
- First, we might care about **streaming the individual messages**, so that would be:
    - the AI message that determines what action to take,
    - the observation message that represents the result of taking that action.
- The second thing we might take care about streaming is **tokens**. So, for each token of the LLM call, we might want to stream the output.

To begin, we're just going to start by streaming the messages. We'll do the tokens later on.

In [11]:
messages = [HumanMessage(content="What is the weather in sf?")]

We're now going to add the concept of a **thread config**, which will be used to keep track of different theads, inside the persistent checkpointer.

This will allow us to **have multiple conversations going on at the same time**, which is really needed for production applications, where you generally have many users.

This **thread config** is only a `dict`, with a `"configurable"` key. Its value is another `dict` with a lone key being `"thread_id"`, and its value as a `str`, here `"1"` (`uuid`s can also be used.)

In [15]:
thread = {"configurable": {"thread_id": "1"}}

We'll now call the graph, not with `invoke`, but with `stream`, passing:
- the same dictionary,
- `thread` as a second argument.

We're then gonna get back a **stream** of events, which represent **updates to that state over time**.



In [16]:
# Temporarily manage issue with LangSmith (don't bother with this ftm)
!export LANGCHAIN_TRACING_V2="false"

## Initial Stream

In [21]:
for event in abot.graph.stream({"messages": messages}, thread):
    # rprint(event)
    for v in event.values():
        rprint(v["messages"])

[
    AIMessage(
        content='',
        additional_kwargs={
            'tool_calls': [
                {
                    'id': 'call_AOSKlG76P36uUTAwuCHGA3mA',
                    'function': {
                        'arguments': '{"query":"weather in San Francisco"}',
                        'name': 'tavily_search_results_json'
                    },
                    'type': 'function'
                }
            ]
        },
        response_metadata={
            'token_usage': {'completion_tokens': 21, 'prompt_tokens': 2008, 'total_tokens': 2029},
            'model_name': 'gpt-3.5-turbo',
            'system_fingerprint': None,
            'finish_reason': 'tool_calls',
            'logprobs': None
        },
        id='run-7b5bc405-513c-435c-a754-2b31e2977c83-0',
        tool_calls=[
            {
                'name': 'tavily_search_results_json',
                'args': {'query': 'weather in San Francisco'},
                'id': 'call_AOSKlG76P36uUTAwuCHGA3mA'
            }
        ],
        usage_metadata={'input_tokens': 2008, 'output_tokens': 21, 'total_tokens': 2029}
    )
]

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_AOSKlG76P36uUTAwuCHGA3mA'}
Back to the model!


[
    ToolMessage(
        content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'San 
Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': 
-122.42, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1718164021, \'localtime\': \'2024-06-11 20:47\'},
\'current\': {\'last_updated_epoch\': 1718163900, \'last_updated\': \'2024-06-11 20:45\', \'temp_c\': 20.3, 
\'temp_f\': 68.5, \'is_day\': 0, \'condition\': {\'text\': \'Clear\', \'icon\': 
\'//cdn.weatherapi.com/weather/64x64/night/113.png\', \'code\': 1000}, \'wind_mph\': 16.1, \'wind_kph\': 25.9, 
\'wind_degree\': 280, \'wind_dir\': \'W\', \'pressure_mb\': 1010.0, \'pressure_in\': 29.82, \'precip_mm\': 0.0, 
\'precip_in\': 0.0, \'humidity\': 59, \'cloud\': 0, \'feelslike_c\': 20.3, \'feelslike_f\': 68.5, \'windchill_c\': 
14.8, \'windchill_f\': 58.6, \'heatindex_c\': 15.2, \'heatindex_f\': 59.4, \'dewpoint_c\': 10.8, \'dewpoint_f\': 
51.4, \'vis_km\': 16.0, \'vis_miles\': 9.0, \'uv\': 5.0, \'gust_mph\': 20.6, \'gust_kph\': 33.1}}"}, {\'url\': 
\'https://world-weather.info/forecast/usa/san_francisco/december-2024/\', \'content\': \'Extended weather forecast 
in San Francisco. Hourly Week 10 days 14 days 30 days Year. Detailed ⚡ San Francisco Weather Forecast for December
2024 - day/night 🌡️ temperatures, precipitations - World-Weather.info.\'}]',
        name='tavily_search_results_json',
        tool_call_id='call_AOSKlG76P36uUTAwuCHGA3mA'
    )
]

[
    AIMessage(
        content='The current weather in San Francisco is clear with a temperature of 20.3°C (68.5°F). The wind 
speed is 25.9 km/h coming from the west. The humidity is at 59%, and there is no precipitation.',
        response_metadata={
            'token_usage': {'completion_tokens': 52, 'prompt_tokens': 2531, 'total_tokens': 2583},
            'model_name': 'gpt-3.5-turbo',
            'system_fingerprint': None,
            'finish_reason': 'stop',
            'logprobs': None
        },
        id='run-2ccae1cb-ca80-42b1-a116-1f5c0dd190dc-0',
        usage_metadata={'input_tokens': 2531, 'output_tokens': 52, 'total_tokens': 2583}
    )
]

We get back a stream of events:

> - **first, we get an `AIMessage`, which is the first result from the language model**.

```python
AIMessage(
    content='',
    additional_kwargs={
        'tool_calls': [{
            'id': 'call_eqDDo3U0wuFrWTitkRpMCIw4',
            'function': {
                'arguments': '{"query":"weather in San Francisco"}',
                'name': 'tavily_search_results_json'}, 
                'type': 'function'
            }]
        }, 
    response_metadata={
        'token_usage': {'completion_tokens': 21, 'prompt_tokens': 1186, 'total_tokens': 1207}, 
        'model_name': 'gpt-3.5-turbo', 
        'system_fingerprint': None, 
        'finish_reason': 'tool_calls', 
        'logprobs': None
        }, 
    id='run-af070dca-58fc-42d1-9d5b-c2d081e72a86-0', 
    tool_calls=[{
        'name': 'tavily_search_results_json', 
        'args': {'query': 'weather in San Francisco'}, 
        'id': 'call_eqDDo3U0wuFrWTitkRpMCIw4'
        }], 
    usage_metadata={'input_tokens': 1186, 'output_tokens': 21, 'total_tokens': 1207}
)
```
> - **It tells us to call `tavily`, which is automatically logged aftewards with this printing**:

```python
Calling: {
    'name': 'tavily_search_results_json',
    'args': {'query': 'weather in San Francisco'},
    'id': 'call_eqDDo3U0wuFrWTitkRpMCIw4'
}
```
> - **Then, the action is performed, is logged with the printing of `Back to the model!`, and we get the following `ToolMessage` (I won't parse it now as it doesn't really improves readability), which is the result of calling `tavily` and, hence, the result of the search**:

```python
ToolMessage(
    content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1717751204, \'localtime\': \'2024-06-07 2:06\'}, \'current\': {\'last_updated_epoch\': 1717750800, \'last_updated\': \'2024-06-07 02:00\', \'temp_c\': 12.2, \'temp_f\': 54.0, \'is_day\': 0, \'condition\': {\'text\': \'Clear\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/113.png\', \'code\': 1000}, \'wind_mph\': 4.3, \'wind_kph\': 6.8, \'wind_degree\': 10, \'wind_dir\': \'N\', \'pressure_mb\': 1011.0, \'pressure_in\': 29.84, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 93, \'cloud\': 0, \'feelslike_c\': 10.7, \'feelslike_f\': 51.2, \'windchill_c\': 9.8, \'windchill_f\': 49.6, \'heatindex_c\': 11.4, \'heatindex_f\': 52.6, \'dewpoint_c\': 9.3, \'dewpoint_f\': 48.8, \'vis_km\': 14.0, \'vis_miles\': 8.0, \'uv\': 1.0, \'gust_mph\': 14.0, \'gust_kph\': 22.5}}"}, {\'url\': \'https://www.weather.gov/index.php/mtr/\', \'content\': \'Current Conditions showing NA; Customize Your Weather.gov. Enter Your City, ST or ZIP Code ... 2024 at 9:40:09 am PDT Watches, Warnings & Advisories. Zoom Out. Excessive Heat Warning. Gale Warning. Heat Advisory. Small Craft Advisory. ... National Weather Service San Francisco Bay Area, CA 21 Grace Hopper Ave, Stop 5 Monterey, CA 93943-5505\'}]', 
    name='tavily_search_results_json', 
    tool_call_id='call_eqDDo3U0wuFrWTitkRpMCIw4'
)
```
> - **Finally, there's an `AIMessage`, which is the result of the LLM, answering our question**:

```python
AIMessage(
    content='The current weather in San Francisco is clear with a temperature of 54.0°F (12.2°C). The wind is blowing at 4.3 mph from the north, and the humidity is at 93%.', 
    response_metadata={
        'token_usage': {'completion_tokens': 46, 'prompt_tokens': 1732, 'total_tokens': 1778}, 
        'model_name': 'gpt-3.5-turbo', 
        'system_fingerprint': None, 
        'finish_reason': 'stop', 
        'logprobs': None
    }, 
    id='run-de9261f7-2481-4194-b1ac-0e9e5cebaf7b-0', 
    usage_metadata={'input_tokens': 1732, 'output_tokens': 46, 'total_tokens': 1778}
)
```

With this `stream` method, we:
- get back all of these intermediate results, 
- have a good visibility of what exactly is going on.

## Follow-Up Question

Let's now call this stream with another message.

So, this is continuing the same conversation that we had before, with asking a follow-up question.

**We don't say anything about the weather, but based on it being a conversation, we would expect it to realize that we're asking about the weather here**.

To mention it's the same conversation, of course, we're passing here **the same `thread_id`**.

In [23]:
messages = [HumanMessage(content="What about in la?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        rprint(v["messages"])

[
    AIMessage(
        content='',
        additional_kwargs={
            'tool_calls': [
                {
                    'id': 'call_sswMVjt7V78OMyoQcW1B4Y0z',
                    'function': {
                        'arguments': '{"query":"weather in Los Angeles"}',
                        'name': 'tavily_search_results_json'
                    },
                    'type': 'function'
                }
            ]
        },
        response_metadata={
            'token_usage': {'completion_tokens': 21, 'prompt_tokens': 3161, 'total_tokens': 3182},
            'model_name': 'gpt-3.5-turbo',
            'system_fingerprint': None,
            'finish_reason': 'tool_calls',
            'logprobs': None
        },
        id='run-6025562f-0aba-47fc-9d30-3a4fbfc1bbda-0',
        tool_calls=[
            {
                'name': 'tavily_search_results_json',
                'args': {'query': 'weather in Los Angeles'},
                'id': 'call_sswMVjt7V78OMyoQcW1B4Y0z'
            }
        ],
        usage_metadata={'input_tokens': 3161, 'output_tokens': 21, 'total_tokens': 3182}
    )
]

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles'}, 'id': 'call_sswMVjt7V78OMyoQcW1B4Y0z'}
Back to the model!


[
    ToolMessage(
        content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Los 
Angeles\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 34.05, \'lon\': -118.24,
\'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1718164291, \'localtime\': \'2024-06-11 20:51\'}, 
\'current\': {\'last_updated_epoch\': 1718163900, \'last_updated\': \'2024-06-11 20:45\', \'temp_c\': 17.2, 
\'temp_f\': 63.0, \'is_day\': 0, \'condition\': {\'text\': \'Overcast\', \'icon\': 
\'//cdn.weatherapi.com/weather/64x64/night/122.png\', \'code\': 1009}, \'wind_mph\': 3.8, \'wind_kph\': 6.1, 
\'wind_degree\': 270, \'wind_dir\': \'W\', \'pressure_mb\': 1013.0, \'pressure_in\': 29.91, \'precip_mm\': 0.0, 
\'precip_in\': 0.0, \'humidity\': 78, \'cloud\': 100, \'feelslike_c\': 17.2, \'feelslike_f\': 63.0, 
\'windchill_c\': 20.5, \'windchill_f\': 68.8, \'heatindex_c\': 20.9, \'heatindex_f\': 69.7, \'dewpoint_c\': 12.9, 
\'dewpoint_f\': 55.1, \'vis_km\': 16.0, \'vis_miles\': 9.0, \'uv\': 6.0, \'gust_mph\': 7.2, \'gust_kph\': 11.6}}"},
{\'url\': \'https://www.accuweather.com/en/us/los-angeles/90012/december-weather/347625\', \'content\': \'Get the 
monthly weather forecast for Los Angeles, CA, including daily high/low, historical averages, to help you plan 
ahead.\'}]',
        name='tavily_search_results_json',
        tool_call_id='call_sswMVjt7V78OMyoQcW1B4Y0z'
    )
]

[
    AIMessage(
        content='The current weather in Los Angeles is overcast with a temperature of 17.2°C (63.0°F). The wind 
speed is 6.1 km/h coming from the west. The humidity is at 78%, and there is no precipitation.',
        response_metadata={
            'token_usage': {'completion_tokens': 53, 'prompt_tokens': 3663, 'total_tokens': 3716},
            'model_name': 'gpt-3.5-turbo',
            'system_fingerprint': None,
            'finish_reason': 'stop',
            'logprobs': None
        },
        id='run-fabcfb6e-2614-4704-ae77-047f7260e3e7-0',
        usage_metadata={'input_tokens': 3663, 'output_tokens': 53, 'total_tokens': 3716}
    )
]

And we can notice everything works as expected.

## Comparing Previous Answers

In [24]:
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        rprint(v["messages"])

[
    AIMessage(
        content='San Francisco currently has a temperature of 20.3°C (68.5°F) while Los Angeles has a temperature 
of 17.2°C (63.0°F). Therefore, San Francisco is warmer than Los Angeles at the moment.',
        response_metadata={
            'token_usage': {'completion_tokens': 50, 'prompt_tokens': 3728, 'total_tokens': 3778},
            'model_name': 'gpt-3.5-turbo',
            'system_fingerprint': None,
            'finish_reason': 'stop',
            'logprobs': None
        },
        id='run-3a47caab-cd6c-49a4-a445-881bc5b0777c-0',
        usage_metadata={'input_tokens': 3728, 'output_tokens': 50, 'total_tokens': 3778}
    )
]

## Changing `thread_id`

> **NOTE**
> 
> Here...
> - at best, we expect the model to **underline there's a problem of context**.
> - at worst, we expect it to **hallucinate, doing its best to fulfill the request without a necessary context**.

In [25]:
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        rprint(v["messages"])

[
    AIMessage(
        content='',
        additional_kwargs={
            'tool_calls': [
                {
                    'id': 'call_fT3Ga9y9OQdgiaSVbl7thxc2',
                    'function': {
                        'arguments': '{"query": "average temperature in Miami"}',
                        'name': 'tavily_search_results_json'
                    },
                    'type': 'function'
                },
                {
                    'id': 'call_qUCiBFtUvOJABP7ID4cvppEz',
                    'function': {
                        'arguments': '{"query": "average temperature in New York"}',
                        'name': 'tavily_search_results_json'
                    },
                    'type': 'function'
                }
            ]
        },
        response_metadata={
            'token_usage': {'completion_tokens': 58, 'prompt_tokens': 151, 'total_tokens': 209},
            'model_name': 'gpt-3.5-turbo',
            'system_fingerprint': None,
            'finish_reason': 'tool_calls',
            'logprobs': None
        },
        id='run-213bb770-24c1-45da-a546-34b51d5f4dad-0',
        tool_calls=[
            {
                'name': 'tavily_search_results_json',
                'args': {'query': 'average temperature in Miami'},
                'id': 'call_fT3Ga9y9OQdgiaSVbl7thxc2'
            },
            {
                'name': 'tavily_search_results_json',
                'args': {'query': 'average temperature in New York'},
                'id': 'call_qUCiBFtUvOJABP7ID4cvppEz'
            }
        ],
        usage_metadata={'input_tokens': 151, 'output_tokens': 58, 'total_tokens': 209}
    )
]

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'average temperature in Miami'}, 'id': 'call_fT3Ga9y9OQdgiaSVbl7thxc2'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'average temperature in New York'}, 'id': 'call_qUCiBFtUvOJABP7ID4cvppEz'}
Back to the model!


[
    ToolMessage(
        content="[{'url': 'https://www.timeanddate.com/weather/usa/miami/climate', 'content': 'Annual Weather 
Averages Near Miami. Averages are for Miami International Airport, which is 8 miles from Miami. Based on weather 
reports collected during 1992-2021. Showing: All Year Climate & Weather Averages in Miami. High Temp: 91 °F. Low 
Temp: 61 ...'}, {'url': 
'https://weatherspark.com/y/18622/Average-Weather-in-Miami-Florida-United-States-Year-Round', 'content': 'The 
hottest month of the year in Miami is August, with an average high of 89°F and low of 78°F. The cool season lasts 
for 3.0 months, from December 5 to March 5, with an average daily high temperature below 78°F. The coldest month of
the year in Miami is January, with an average low of 63°F and high of 76°F.'}]",
        name='tavily_search_results_json',
        tool_call_id='call_fT3Ga9y9OQdgiaSVbl7thxc2'
    ),
    ToolMessage(
        content='[{\'url\': \'https://www.usclimatedata.com/climate/new-york/united-states/3202\', \'content\': 
\'Station Data. Monthly averages New York Longitude: -74, Latitude: 40.69 Average weather New York, NY - 11201. 
Monthly: 1981-2010 normals\'}, {\'url\': \'https://www.timeanddate.com/weather/usa/new-york/climate\', \'content\':
\'All Year Climate & Weather Averages in New York\\nHigh Temp:\\n85\\xa0°F\\nLow Temp:\\n27\\xa0°F\\nMean 
Temp:\\n55\\xa0°F\\nPrecipitation:\\n2.03"\\nHumidity:\\n62%\\nDew Point:\\n42\\xa0°F\\nWind:\\n6 
mph\\nPressure:\\n30.04 "Hg\\nVisibility:\\n10\\xa0mi\\nJanuary Climate & Weather Averages in New York\\nHigh 
Temp:\\n40\\xa0°F\\nLow Temp:\\n27\\xa0°F\\nMean Temp:\\n34\\xa0°F\\nPrecipitation:\\n1.98"\\nHumidity:\\n61%\\nDew
Point:\\n21\\xa0°F\\nWind:\\n8 mph\\nPressure:\\n30.08 "Hg\\nVisibility:\\n10\\xa0mi\\nFebruary Climate & Weather 
Averages in New York\\nHigh Temp:\\n42\\xa0°F\\nLow Temp:\\n28\\xa0°F\\nMean 
Temp:\\n35\\xa0°F\\nPrecipitation:\\n1.47"\\nHumidity:\\n57%\\nDew Point:\\n21\\xa0°F\\nWind:\\n8 
mph\\nPressure:\\n30.06 "Hg\\nVisibility:\\n10\\xa0mi\\nMarch Climate & Weather Averages in New York\\nHigh 
Temp:\\n50\\xa0°F\\nLow Temp:\\n35\\xa0°F\\nMean Temp:\\n43\\xa0°F\\nPrecipitation:\\n2.08"\\nHumidity:\\n57%\\nDew
Point:\\n27\\xa0°F\\nWind:\\n8 mph\\nPressure:\\n30.06 "Hg\\nVisibility:\\n10\\xa0mi\\nApril Climate & Weather 
Averages in New York\\nHigh Temp:\\n62\\xa0°F\\nLow Temp:\\n45\\xa0°F\\nMean 
Temp:\\n53\\xa0°F\\nPrecipitation:\\n2.04"\\nHumidity:\\n57%\\nDew Point:\\n37\\xa0°F\\nWind:\\n7 
mph\\nPressure:\\n30.00 "Hg\\nVisibility:\\n10\\xa0mi\\nMay Climate & Weather Averages in New York\\nHigh 
Temp:\\n72\\xa0°F\\nLow Temp:\\n54\\xa0°F\\nMean Temp:\\n63\\xa0°F\\nPrecipitation:\\n2.27"\\nHumidity:\\n63%\\nDew
Point:\\n49\\xa0°F\\nWind:\\n6 mph\\nPressure:\\n30.00 "Hg\\nVisibility:\\n9\\xa0mi\\nJune Climate & Weather 
Averages in New York\\nHigh Temp:\\n80\\xa0°F\\nLow Temp:\\n64\\xa0°F\\nMean 
Temp:\\n72\\xa0°F\\nPrecipitation:\\n2.47"\\nHumidity:\\n65%\\nDew Point:\\n59\\xa0°F\\nWind:\\n5 
mph\\nPressure:\\n29.96 "Hg\\nVisibility:\\n9\\xa0mi\\nJuly Climate & Weather Averages in New York\\nHigh 
Temp:\\n85\\xa0°F\\nLow Temp:\\n69\\xa0°F\\nMean Temp:\\n77\\xa0°F\\nPrecipitation:\\n2.36"\\nHumidity:\\n64%\\nDew
Point:\\n64\\xa0°F\\nWind:\\n4 mph\\nPressure:\\n29.97 "Hg\\nVisibility:\\n9\\xa0mi\\nAugust Climate & Weather 
Averages in New York\\nHigh Temp:\\n84\\xa0°F\\nLow Temp:\\n68\\xa0°F\\nMean 
Temp:\\n76\\xa0°F\\nPrecipitation:\\n2.02"\\nHumidity:\\n66%\\nDew Point:\\n63\\xa0°F\\nWind:\\n5 
mph\\nPressure:\\n30.01 "Hg\\nVisibility:\\n9\\xa0mi\\nSeptember Climate & Weather Averages in New York\\nHigh 
Temp:\\n76\\xa0°F\\nLow Temp:\\n61\\xa0°F\\nMean Temp:\\n69\\xa0°F\\nPrecipitation:\\n1.85"\\nHumidity:\\n67%\\nDew
Point:\\n57\\xa0°F\\nWind:\\n5 mph\\nPressure:\\n30.06 "Hg\\nVisibility:\\n10\\xa0mi\\nOctober Climate & Weather 
Averages in New York\\nHigh Temp:\\n64\\xa0°F\\nLow Temp:\\n50\\xa0°F\\nMean 
Temp:\\n57\\xa0°F\\nPrecipitation:\\n1.77"\\nHumidi

[
    AIMessage(
        content='The average temperature in Miami is around 89°F in August, which is one of the hottest months. In 
comparison, the average temperature in New York is around 85°F in July, which is one of the warmest months. Based 
on this information, Miami tends to be slightly warmer than New York on average.',
        response_metadata={
            'token_usage': {'completion_tokens': 65, 'prompt_tokens': 2219, 'total_tokens': 2284},
            'model_name': 'gpt-3.5-turbo',
            'system_fingerprint': None,
            'finish_reason': 'stop',
            'logprobs': None
        },
        id='run-57266e7f-2b7e-4d40-99aa-88f29e1777d3-0',
        usage_metadata={'input_tokens': 2219, 'output_tokens': 65, 'total_tokens': 2284}
    )
]

And indeed, we get the worst scenario, with the model doing its best to get two measures to compare, and getting (without us being able to know why...) temperatures in Miami and New York.

> **NOTE**
>
> For comparison purposes, **it would be interesting to see how a more recent model like -4o would perform**...

# Streaming Tokens

In [26]:
try:
    from langgraph.checkpoint.aiosqlite import AsyncSqliteSaver
except ImportError as e:
    print(e)

If the previous code doesn't work, it can be fixed with installing [**aiosqlite**](https://pypi.org/project/aiosqlite/), as mentioned in [**AsyncSqliteSaver's documentation**](https://langchain-ai.github.io/langgraph/reference/checkpoints/#asyncsqlitesaver).

In [28]:
memory = AsyncSqliteSaver.from_conn_string(":memory:")

We're gonna use a new `"thread_id"` to start a brand new conversation.

We're also going to be iterating over a different type of event. These events represent updates from the underlying stream.

We want to **look for events that correspond to new tokens**.

These kind of events are called `"on_chat_model_stream"`.

When we see these events happening, we want to get the content, and print it out with a pipe delimiter.

When running this, we should see it streaming real time into the screen.

In [29]:
messages = [HumanMessage(content="What is the weather in SF?")]
thread = {"configurable": {"thread_id": "4"}}
async for event in abot.graph.astream_events({"messages": messages}, thread, version="v1"):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")

NotImplementedError: The SqliteSaver does not support async methods. Consider using AsyncSqliteSaver instead.
from langgraph.checkpoint.aiosqlite import AsyncSqliteSaver
Note: AsyncSqliteSaver requires the aiosqlite package to use.
Install with:
`pip install aiosqlite`
See https://langchain-ai.github.io/langgraph/reference/checkpoints/#asyncsqlitesaverfor more information.